### Aggregation :sum(),mean(),median() and max()

In [3]:
import pandas as pd
import numpy as np

rng=np.random.RandomState(42)
ser=pd.Series(rng.rand(5))
ser

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

In [4]:
ser.sum()

2.811925491708157

In [5]:
ser.mean()

0.5623850983416314

In [6]:
# For a DateFrame,by default aggregates return result within each column

In [7]:
df = pd.DataFrame({'A': rng.rand(5),
                   'B': rng.rand(5)})
df


,A,B
0,0.155995,0.020584
1,0.058084,0.969910
2,0.866176,0.832443
3,0.601115,0.212339
4,0.708073,0.181825


In [8]:
df.mean()

A    0.477888
B    0.443420
dtype: float64

In [9]:
# define axis column to specify the axis argument
df.mean(axis='columns')

0    0.088290
1    0.513997
2    0.849309
3    0.406727
4    0.444949
dtype: float64

In [10]:
df.describe()

,A,B
count,5.000000,5.000000
mean,0.477888,0.443420
std,0.353125,0.426952
min,0.058084,0.020584
25%,0.155995,0.181825
50%,0.601115,0.212339
75%,0.708073,0.832443
max,0.866176,0.969910


## Group By : split ,Apply,combine

In [12]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [19]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


## aggregate,filter,transform,apply

In [21]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                  columns = ['key', 'data1', 'data2'])
df


,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [23]:
# aggregation

df.groupby('key').aggregate([min,np.median,max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

In [26]:
df.groupby('key').aggregate({'data1':'min', 'data2':'min'})

,data1,data2
key,,
A,0,3
B,1,0
C,2,3


In [27]:
# filtering
def filter_func(x):
    return x['data2'].std() >4

df.groupby('key').std()

,data1,data2
key,,
A,2.12132,1.414214
B,2.12132,4.949747
C,2.12132,4.242641


In [30]:
df.groupby('key').filter(filter_func)  #std greater than 4 show

,key,data1,data2
1,B,1,0
2,C,2,3
4,B,4,7
5,C,5,9


In [32]:
# transformation
df.groupby('key').transform(lambda x:x-x.mean())

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


In [35]:
# the apply() method
def norm_by_data2(x):
        # x is a DataFrame of group values
        x['data1'] /= x['data2'].sum()
        return x
    
print(df);
print(df.groupby('key').apply(norm_by_data2))


  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9
  key     data1  data2
0   A  0.000000      5
1   B  0.142857      0
2   C  0.166667      3
3   A  0.375000      3
4   B  0.571429      7
5   C  0.416667      9


## specifying the split key

In [37]:
L=[0,1,0,1,2,0]
df.groupby(L).sum()

,data1,data2
0,7,17
1,4,3
2,4,7


In [40]:
df.groupby(df['key']).sum()

,data1,data2
key,,
A,3,8
B,5,7
C,7,12


## a dictionary or series mapping index to group


In [41]:
df2=df.set_index('key')
df2

,data1,data2
key,,
A,0,5
B,1,0
C,2,3
A,3,3
B,4,7
C,5,9


In [42]:
mapping={'A':'vowel','B':'consonant','C':'consonant'}
mapping

{'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}

In [44]:
df2.groupby(mapping).sum()

,data1,data2
consonant,12,19
vowel,3,8


In [46]:
## any python function
df2.groupby(str.lower).mean()

,data1,data2
a,1.5,4.0
b,2.5,3.5
c,3.5,6.0


In [47]:
# a list of valid keys
df2.groupby([str.lower,mapping]).mean()

,,data1,data2
a,vowel,1.5,4.0
b,consonant,2.5,3.5
c,consonant,3.5,6.0
